In [4]:
%%writefile lab/gaussian_elimination.cpp

#include <iostream>
#include <fstream>
#include <chrono>
#include <ratio>
#include <iomanip>
#include <random>
#include <thread>
#include <functional>

#include <CL/sycl.hpp>

using namespace cl::sycl;

void matrix_print(buffer<float, 2>& buf);

void matrix_copy(buffer<float, 2>& to, buffer<float, 2>& from);

void matrix_init(buffer<float, 2>& buf);

// 高斯消元算法的串行化实现
void gaussian_elimination_serial(buffer<float, 2>& buf, queue& q);

// 高斯消元算法的并行化实现
void gaussian_elimination_parallel(buffer<float, 2>& buf, queue& q);

using gauss_func = std::function<void(buffer<float, 2>&, queue&)>;

double test(int n, const gauss_func& gauss, int times, queue& q);

void normal_test_output(const std::vector<gauss_func>& gauss_funcs, const std::vector<std::string>& names, int times, int begin, int end, queue& q);

int main() {

    queue q;
    device my_device = q.get_device();
    std::cout << "Device: " << my_device.get_info<info::device::name>() << std::endl;

    std::vector<gauss_func> gauss_funcs = {
            gaussian_elimination_serial,
            gaussian_elimination_parallel,
    };

    std::vector<std::string> names = {
            "串行结果",
            "并行结果",
    };

    const int times = 10;
    const int begin = 16;
    const int end = 1024;

    normal_test_output(gauss_funcs, names, times, begin, end, q);
}

void matrix_print(buffer<float, 2>& buf){
    host_accessor m{ buf ,read_only };
    auto range = m.get_range();
    for (int i = 0; i < range[0]; i++) {
        for (int j = 0; j < range[1]; j++) {
            std::cout << std::setw(16) << m[i][j];
        }
        std::cout << std::endl;
    }
}

void matrix_copy(buffer<float, 2>& to, buffer<float, 2>& from){
    host_accessor ms{ from ,read_only };
    host_accessor md{ to ,write_only };
    assert(ms.get_range() == md.get_range());
    auto range = ms.get_range();
    for (int i = 0; i < range[0]; i++) {
        for (int j = 0; j < range[1]; j++) {
            md[i][j] = ms[i][j];
        }
    }
}

void matrix_init(buffer<float, 2>& buf){
    host_accessor m{ buf ,read_write };

    static std::default_random_engine generator(1337);
    static std::uniform_real_distribution<float> distribution(-1.0, 1.0);

    int n = m.get_range()[0];
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < i; j++)
            m[i][j] = 0;
        m[i][i] = 1.0;
        for (int j = i + 1; j < n; j++)
            m[i][j] = distribution(generator);
    }
    for (int k = 0; k < n; k++) {
        for (int i = k + 1; i < n; i++) {
            for (int j = 0; j < n; j++) {
                m[i][j] += m[k][j];
            }
        }
    }
}

// 高斯消元算法的串行化实现
void gaussian_elimination_serial(buffer<float, 2>& buf, queue& q){
    host_accessor m{ buf ,read_write };
    int n = m.get_range()[0];
    for (int k = 0; k < n; k++) {
        for (int j = k + 1; j < n; j++) {
            m[k][j] = m[k][j] / m[k][k];
        }
        m[k][k] = 1;
        for (int i = k + 1; i < n; i++) {
            for (int j = k + 1; j < n; j++) {
                m[i][j] = m[i][j] - m[i][k] * m[k][j];
            }
            m[i][k] = 0;
        }
    }
}

// 高斯消元算法的并行化实现
void gaussian_elimination_parallel(buffer<float, 2>& buf, queue& q){
    int n = buf.get_range()[0];
    for (int k = 0; k < n; k++) {

        q.submit([&](handler& h) {
            accessor m{ buf, h, read_write };
            h.parallel_for(range(n - k), [=](auto idx) {
                int j = k + idx;
                m[k][j] = m[k][j] / m[k][k];
                });
            });

        q.submit([&](handler& h) {
            accessor m{ buf, h, read_write };
            h.parallel_for(range(n - (k + 1), n - (k + 1)), [=](auto idx) {
                int i = k + 1 + idx.get_id(0);
                int j = k + 1 + idx.get_id(1);
                m[i][j] = m[i][j] - m[i][k] * m[k][j];
                });
            });

        q.submit([&](handler& h) {
            accessor m{ buf, h, read_write };
            h.parallel_for(range(n - (k + 1)), [=](auto idx) {
                int i = k + 1 + idx;
                m[i][k] = 0;
                });
            });
    }
    q.wait();
}

double test(int n, const gauss_func& gauss, int times, queue& q){
    buffer<float, 2> buf(range(n, n));

    // warm up
    matrix_init(buf);
    gauss(buf, q);

    std::chrono::duration<double, std::milli> elapsed{};
    for (int i = 0; i < times; i++) {
        matrix_init(buf);
        auto start = std::chrono::high_resolution_clock::now();
        gauss(buf,q);
        auto end = std::chrono::high_resolution_clock::now();
        elapsed += end - start;
    }
    return elapsed.count() / times;
}

void normal_test_output(const std::vector<gauss_func>& gauss_funcs, 
                        const std::vector<std::string>& names, 
                        int times, 
                        int begin, 
                        int end, 
                        queue& q){
    //generate the table bar
    std::cout << "规模,";
    for (auto& name : names) {
        std::cout << name << ",";
    }
    std::cout << std::endl;

    //generate the table content
    for (int n = begin; n <= end; n *= 2) {
        std::cout << n << ",";
        for (auto& func : gauss_funcs) {
            std::cout << test(n, func, times, q) << ",";
        }
        std::cout << std::endl;
    }
}

Writing lab/gaussian_elimination.cpp


In [5]:
%pycat compile_gaussian_elimination.sh

#!/bin/bash
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1
/bin/echo "##" $(whoami) is compiling
icpx -fsycl lab/gaussian_elimination.cpp -o gaussian_elimination -O3 -Wall
if [ $? -eq 0 ]; then ./gaussian_elimination; fi


In [6]:
! chmod +x q compile_gaussian_elimination.sh;if [ -x "$(command -v qsub)" ]; then ./q compile_gaussian_elimination.sh; else compile_gaussian_elimination.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2107350.v-qsvr-1           ...ub-singleuser u175645         00:00:12 R jupyterhub     
2107421.v-qsvr-1           ...limination.sh u175645                0 Q batch          

Waiting for Output ████████████████████████████████████████████████████████████

TimeOut 60 seconds: Job is still queued for execution, check for output file later (compile_gaussian_elimination.sh.o2107421)

